In [9]:
import requests
import json
import os, re, unicodedata
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from notebookutils import mssparkutils

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 11, Finished, Available, Finished)

In [10]:
# ---------- Spark ----------
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 12, Finished, Available, Finished)

In [11]:
# Seleccionar el Lakehouse destino (bronze)
spark.sql("USE lh_bronze")

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 13, Finished, Available, Finished)

DataFrame[]

In [12]:
headers = {
    "Accept": "application/json; application/vnd.esios-api-v1+json",
    "Content-Type": "application/json",
    "x-api-key": "273695bb4f30b8ab8de054a0410ee2a9d09c97a5006e3a07c27615e6c60888a7"
}

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 14, Finished, Available, Finished)

In [13]:
# 📌 Indicadores con sus nombres
indicators = [
    {"type": "demanda","id": "10339"}
]

# 📆 Rango de fechas por año
date_ranges = {
    "2023": ("01-01-2023T00:00", "31-12-2023T23:55"),
    "2024": ("01-01-2024T00:00", "31-12-2024T23:55"),
    "2025": ("01-01-2025T00:00", "30-9-2025T23:55"),
}

# ⏱️ Granularidades
time_truncs = ["month"] # ["day", "month"] #solo muestra por meses

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 15, Finished, Available, Finished)

In [14]:
# 📂 Carpeta base dentro del Lakehouse (BRONZE)
base_dir = "/lakehouse/default/Files/bronze/ESIOS/data/demanda/evolucion"

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 16, Finished, Available, Finished)

In [15]:
# 📂 Salida (Tables/)
tables_prefix = "Tables"

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 17, Finished, Available, Finished)

In [16]:
# 🚀 Loop de descargas
for indicator in indicators:
    for year, (start_date, end_date) in date_ranges.items():
        for trunc in time_truncs:
            url = (
                f"https://api.esios.ree.es/indicators/{indicator['id']}?"
                f"start_date={start_date}&end_date={end_date}&time_trunc={trunc}"
            )
            try:
                response = requests.get(url, headers=headers, timeout=60)
                response.raise_for_status()
                data = response.json()

                # ----------------------
                # 1) Guardar JSON crudo en Files/
                # ----------------------
                filename = f"brz-{indicator['type']}-{trunc}-{year}.json"
                year_dir = os.path.join(base_dir, trunc, str(year))
                os.makedirs(year_dir, exist_ok=True)
                filepath = os.path.join(year_dir, filename)

                with open(filepath, "w", encoding="utf-8") as f:
                    json.dump(data, f, ensure_ascii=False, indent=2)

                print(f"✅ Guardado JSON en Lakehouse: {filepath}")

            except requests.exceptions.RequestException as e:
                print(f"❌ Error en {url}: {e}")

print(f"\nProceso terminado - {datetime.utcnow().isoformat()}Z")

StatementMeta(, 8da818bb-f14a-45fc-a026-3c8d5cde4777, 18, Finished, Available, Finished)

✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/demanda/evolucion/month/2023/brz-demanda-month-2023.json
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/demanda/evolucion/month/2024/brz-demanda-month-2024.json
✅ Guardado JSON en Lakehouse: /lakehouse/default/Files/bronze/ESIOS/data/demanda/evolucion/month/2025/brz-demanda-month-2025.json

Proceso terminado - 2025-10-06T10:28:39.639673Z
